In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [2]:
us = pd.read_csv('USArrests.csv',index_col=0)

In [3]:
us.head()

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


In [6]:
scaler = StandardScaler().set_output(transform='pandas')
us_scaled = scaler.fit_transform(us)
clust_DB = DBSCAN(eps=1,min_samples=2)
clust_DB.fit(us_scaled)
silhouette_score(us_scaled,clust_DB.labels_)

0.19290975823254836

In [15]:
eps_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(us_scaled.iloc[:,:4])
        if len(set(clust_DB.labels_)) > 2:
            cnt = cnt + 1
            us_scaled['Clust'] = clust_DB.labels_
            us_scl_inliers = us_scaled[us_scaled['Clust']!=-1]
            sil_sc = silhouette_score(us_scl_inliers.iloc[:,:-1],
                             us_scl_inliers.iloc[:,-1])
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]


0.4 2 0.9294801120696734
0.6 2 0.4491069431785521
0.6 3 0.6443540494228596
1 2 0.3395677965591881
1 3 0.3395677965591881
1 4 0.4160914171480962
1 5 0.2606933648423083
Best Paramters:


,Sr,eps,min_pt,sil
0,1.0,0.4,2.0,0.92948


#### DBSCAN with best params

In [16]:
clust_DB = DBSCAN(eps=0.4,min_samples=2)
clust_DB.fit(us_scaled)
print(clust_DB.labels_)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1  1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1]


In [17]:
df_clust = pd.DataFrame({'city':list(us_scaled.index),'cluster':list(clust_DB.labels_)})

In [18]:
df_clust.sort_values('cluster')

,city,cluster
0,Alabama,-1
26,Nebraska,-1
27,Nevada,-1
29,New Jersey,-1
30,New Mexico,-1
32,North Carolina,-1
33,North Dakota,-1
34,Ohio,-1
35,Oklahoma,-1
36,Oregon,-1


## Cluster analysis

In [19]:
us_clust = us.copy()
us_clust['cluster'] = clust_DB.labels_
us_clust = us_clust[us_clust['cluster']!=-1]
us_clust.groupby('cluster').mean()

,Murder,Assault,UrbanPop,Rape
cluster,,,,
0,10.75,251.5,84.5,25.05
1,2.15,56.5,56.5,10.40


In [21]:
us_clust.sort_values('cluster')

,Murder,Assault,UrbanPop,Rape,cluster
Illinois,10.4,249,83,24.0,0
New York,11.1,254,86,26.1,0
Iowa,2.2,56,57,11.3,1
New Hampshire,2.1,57,56,9.5,1
